Q1) 이 경우에서는 Type 1 error가 높을까요? Type 2 Error가 높을까요? FP, FN과 관련지어 설명해주세요.

Threshold를 0.3으로 낮춘다는 것은 True라고 판정하는 경우가 더 많아지는 것이기 때문에 False임에도 True라고 판정하는 FP가 더 많아질 것이다. 이는 True가 대립가설이라고 한다면 귀무가설이 맞음에도 불구하고 대립가설을 채택하는 Type 1 Error가 커지는 것이다. 

Q2-1) Accuracy, Precision and Recall이 무엇인지 TP,FP,FN,TN의 식으로 나타내 주세요.

Accuracy = (TP + TN) / (TP + TN + FP + FN)
전체에서 Positive와 Negative를 얼마나 정확히 예측했는지의 여부를 측정

Precision(PPV_Positive Predictive Value) = TP / (TP + FP)
Positive로 예측한 것들 중 실제로 Positive인 비율

Recall(Sensitivity, TPR_True Positive Rate) = TP / (TP + FN)
실제 Positive인 것들 중 Positive로 예측된 비율



Q2-2) Precision and Recall에 관한 제가 든 예시가 아닌 실생활 예시 하나를 들어주세요.

은행이 대출 고객의 부도 여부를 예측하여 대출 가능 여부를 결정할 때 두 개를 모두 고려하여야 한다. 고객을 우량 고객과 불량 고객으로 나눌 때, 실제로는 불량 고객인데 우량 고객이라고 판단하는 경우(FP)가 늘어나게 되면 Precision이 감소하게 된다. 이 경우는 은행이 대출을 승인하게 되면 대출 금액을 회수하지 못하게 될 가능성이 크므로 은행에 타격을 주게 된다. 반대로, 실제로는 우량 고객이지만 불량 고객이라고 판단하는 경우(FN)가 늘어나면 Recall이 감소하게 된다. 이 경우 은행은 우량 고객들에게도 대출을 해주지 않으므로 대출 이자 수익이 감소하게 된다. 따라서 둘 간의 적절한 균형점을 찾는 것이 중요하다.

Q3) 코드 따라해보고 주석 달기!

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [2]:
# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data', train = True, transform = transforms.ToTensor(), download = True) 
  
test_dataset = dsets.MNIST(root ='./data', train = False, transform = transforms.ToTensor()) 

# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False) 

In [4]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [5]:
model = LogisticRegression(input_size, num_classes) 

In [6]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 

In [7]:
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data)) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2013
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1335
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0561
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9201
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.9128
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7493
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7105
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6116
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.6592
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5746
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5829
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.5034
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.4786
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4779
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3165
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.3071
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2100
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2629
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.1559
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1699
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [8]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  82 %


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Q4-1) 2-3 In[14] 코드에서 optim.SGD를 사용하지않고 코드를 짠다면 어떤 방식으로 짜야할까요? 설명해주세요.

optim.SGD를 쓰지 않고 코드를 짜려면 전체 데이터셋을 batch로 나누고 각각에 대해서 gradient를 계산해준 다음 Gradient Descent Algorithm을 구현해주면 된다. 그러나 이 경우에는 데이터가 별로 없기 때문에 batch를 나누는 의미가 없기 때문에 그냥 Gradient Descent Algorithm을 적용해주면 된다.

Q4-2) Gradient descent 코드를 구현하는 문제입니다. 2-3 In[14]의 코드에서 다른 optimizer(3개 ex)adam, rmsprop, sgd에 momentum 추가 등등)를 이용하여 결과값을 비교해주세요.

In [9]:
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1)
  def forward(self, x):
    return self.linear(x)

# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)
  # cost 계산
  cost = F.mse_loss(prediction, y_train)
  # cost로 H(x) 개선
  optimizer.zero_grad() # 미분값이
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    params = list(model.parameters())
    W = params[0].item()
    b = params[1].item()
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.236, b: -0.281 Cost: 3.783830
Epoch  100/1000 W: 0.839, b: 0.303 Cost: 0.017979
Epoch  200/1000 W: 0.875, b: 0.277 Cost: 0.011186
Epoch  300/1000 W: 0.897, b: 0.229 Cost: 0.007645
Epoch  400/1000 W: 0.918, b: 0.182 Cost: 0.004806
Epoch  500/1000 W: 0.938, b: 0.138 Cost: 0.002793
Epoch  600/1000 W: 0.954, b: 0.101 Cost: 0.001504
Epoch  700/1000 W: 0.968, b: 0.072 Cost: 0.000752
Epoch  800/1000 W: 0.978, b: 0.049 Cost: 0.000348
Epoch  900/1000 W: 0.986, b: 0.032 Cost: 0.000150
Epoch 1000/1000 W: 0.991, b: 0.020 Cost: 0.000059


In [10]:
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adamax(model.parameters(), lr=0.01)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)
  # cost 계산
  cost = F.mse_loss(prediction, y_train)
  # cost로 H(x) 개선
  optimizer.zero_grad() # 미분값이
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    params = list(model.parameters())
    W = params[0].item()
    b = params[1].item()
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: -0.002, b: 0.289 Cost: 3.729678
Epoch  100/1000 W: 0.535, b: 0.789 Cost: 0.166727
Epoch  200/1000 W: 0.627, b: 0.806 Cost: 0.096437
Epoch  300/1000 W: 0.659, b: 0.755 Cost: 0.083086
Epoch  400/1000 W: 0.685, b: 0.698 Cost: 0.070789
Epoch  500/1000 W: 0.712, b: 0.638 Cost: 0.059296
Epoch  600/1000 W: 0.739, b: 0.579 Cost: 0.048743
Epoch  700/1000 W: 0.766, b: 0.519 Cost: 0.039242
Epoch  800/1000 W: 0.792, b: 0.460 Cost: 0.030872
Epoch  900/1000 W: 0.818, b: 0.403 Cost: 0.023674
Epoch 1000/1000 W: 0.843, b: 0.348 Cost: 0.017648


In [11]:
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.RMSprop(model.parameters(), lr=0.01)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)
  # cost 계산
  cost = F.mse_loss(prediction, y_train)
  # cost로 H(x) 개선
  optimizer.zero_grad() # 미분값이
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    params = list(model.parameters())
    W = params[0].item()
    b = params[1].item()
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: -0.084, b: -0.809 Cost: 11.675529
Epoch  100/1000 W: 0.897, b: 0.152 Cost: 0.010511
Epoch  200/1000 W: 0.934, b: 0.147 Cost: 0.003138
Epoch  300/1000 W: 0.951, b: 0.108 Cost: 0.001711
Epoch  400/1000 W: 0.971, b: 0.065 Cost: 0.000628
Epoch  500/1000 W: 0.987, b: 0.028 Cost: 0.000120
Epoch  600/1000 W: 0.997, b: 0.007 Cost: 0.000008
Epoch  700/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.017, b: 0.017 Cost: 0.002456


Adamax가 제일 느리게 최적화되고있고, 그 다음 Adam이 느리게 최적화되는 중이다. RMSprop은 700/1000에 최적화를 달성하였으나 1000/1000에서 다시 최적화에서 벗어나는 것을 관찰할 수 있다.